## Target Variable (to be derived):
- *Current Loan Delinquency Status*: The number of days the borrower is delinquent in making loan payments as of the end of the monthly reporting period. Used to derive the target (e.g., delinquent if >0). Guide notes: 0 = Current, 1 = 30-59 days, 2 = 60-89 days, ..., RA = Repayment Plan, RF = REO, 999 = Unknown.

## Predictor Variables (Features):
#### From Origination Data:
- *Credit Score*: The standardized credit score used to evaluate the borrower during the loan origination process. Lower scores indicate higher risk. Guide notes: FICO score, masked as 300 for <300, 850 for >850, or 9999 for missing.
- *Original Combined Loan-to-Value (CLTV)*: The ratio of the original loan amount and any subordinate lien amount to the property value at origination. Higher ratios increase default risk. Guide notes: Rounded to nearest integer, 999 for missing. 
- *Original Debt-to-Income (DTI) Ratio*: : The ratio of the borrower's total monthly debt payments to gross monthly income at origination. Higher DTI suggests financial strain. Guide notes: Rounded to nearest integer, 999 for missing or not considered.
- *Original Interest Rate*: The interest rate on the loan as stated on the note at the time the loan was originated. Higher rates may lead to higher payments and defaults. Guide notes: Reported to the nearest eighth of a percent.
- *Original Loan Term*: The number of months in which the loan is scheduled to be repaid. Longer terms may reduce monthly payments but increase long-term risk. Guide notes: In months, e.g., 360 for 30-year loans.
- *Number of Borrowers*: The number of borrowers who are obligated to repay the mortgage note. Multiple borrowers may reduce risk. Guide notes: 99 for missing.
- *Property State*: The two-letter postal abbreviation for the state in which the property is located. Captures regional economic factors. Guide notes: U.S. states only.
- *Occupancy Status*: The classification for the property occupancy status at the time the loan was originated. Investment properties have higher risk. Guide notes: O = Owner Occupied, S = Second Home, I = Investment Property, 9 = Unknown.

#### From Performance Data:
- *Loan Age*: The number of scheduled monthly payments that have elapsed since the loan was originated. Helps capture loan seasoning. Guide notes: In months, 999 for missing.
- *Remaining Months to Legal Maturity*: The number of months remaining until the loan is scheduled to mature. Shorter terms may indicate higher risk near maturity. Guide notes: In months, 999 for missing.
- *Current Actual UPB*:The unpaid principal balance of the loan as of the end of the monthly reporting period. Higher UPB may correlate with defaults. Guide notes: Rounded to nearest $1,000, 000000 for zero balance.
- *Current Interest Rate*: The interest rate on the loan as of the end of the monthly reporting period. Adjustments can affect affordability. Guide notes: Reported to the nearest eighth of a percent, 99.999 for missing.

Rationale for Selection:<br>
<br>
These variables cover borrower creditworthiness, loan affordability, property details, and ongoing performance, which are key drivers of default risk. The target is derived from 'Current Loan Delinquency Status' as a binary flag (1 for delinquent, 0 for current).<br>
<br>
Key Identifiers:<br>
<br>
- *Loan Sequence Number*: A unique identifier for each loan, critical for merging and tracking across origination and performance data. Guide notes: 12-character alphanumeric, masked for privacy.
- *Original Loan-to-Value (LTV)*: The ratio of the original loan amount to the property value at origination, providing additional context to Original Combined Loan-to-Value (CLTV). Guide notes: Rounded to nearest integer, 999 for missing.
- *First Payment Date*: The date of the first scheduled payment, offering a temporal anchor for loan age and performance. Guide notes: Format YYYYMMDD, parsed as datetime64[ns].

In [12]:
import pandas as pd
import numpy as np
import os

In [13]:
df1 = pd.read_csv("/Users/61310joy/Default_Predict/Data/raw/merged_loans_part_2.csv")
df2 = pd.read_csv("/Users/61310joy/Default_Predict/Data/raw/merged_loans_part_3.csv")

/var/folders/qw/h2s12w2524z6vtfqym33s89m0000gn/T/ipykernel_76755/209006984.py:1: DtypeWarning: Columns (26,28,34,54,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("/Users/61310joy/Default_Predict/Data/raw/merged_loans_part_2.csv")
/var/folders/qw/h2s12w2524z6vtfqym33s89m0000gn/T/ipykernel_76755/209006984.py:2: DtypeWarning: Columns (34,37,38,54,55,59,60) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("/Users/61310joy/Default_Predict/Data/raw/merged_loans_part_3.csv")


In [14]:
df1.head()

,Unnamed: 0,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),...,Modification Cost,Step Modification Flag,Deferred Payment Plan,Estimated Loan-to-Value (ELTV),Zero Balance Removal UPB,Delinquent Accrued Interest,Delinquency Due to Disaster,Borrower Assistance Status Code,Current Month Modification Cost,Interest Bearing UPB
0,F19Q20251019,800,2019-08-01,N,2049-07-01,40900.0,0,1,P,80,...,NaN,NaN,NaN,999,344607.98,NaN,NaN,NaN,NaN,0.00
1,F19Q20251020,795,2019-08-01,Y,2049-07-01,17020.0,0,1,P,80,...,NaN,NaN,NaN,999,311767.76,NaN,NaN,NaN,NaN,0.00
2,F19Q20251021,792,2019-08-01,N,2049-07-01,42644.0,25,1,S,90,...,NaN,NaN,NaN,999,441766.74,NaN,NaN,NaN,NaN,0.00
3,F19Q20251022,737,2019-08-01,N,2049-07-01,NaN,0,1,P,72,...,NaN,NaN,NaN,999,357089.47,NaN,NaN,NaN,NaN,0.00
4,F19Q20251023,738,2019-08-01,N,2049-07-01,36084.0,25,1,P,92,...,NaN,NaN,NaN,64,NaN,NaN,NaN,NaN,NaN,312372.79


In [15]:
df2.head()

,Unnamed: 0,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),...,Modification Cost,Step Modification Flag,Deferred Payment Plan,Estimated Loan-to-Value (ELTV),Zero Balance Removal UPB,Delinquent Accrued Interest,Delinquency Due to Disaster,Borrower Assistance Status Code,Current Month Modification Cost,Interest Bearing UPB
0,F21Q30383329,739,2021-10-01,N,2036-09-01,NaN,0,1,P,63,...,NaN,NaN,NaN,38,NaN,NaN,NaN,NaN,NaN,93595.50
1,F21Q30383330,712,2021-10-01,N,2036-09-01,24300.0,0,1,P,41,...,NaN,NaN,NaN,27,NaN,NaN,NaN,NaN,NaN,117346.65
2,F21Q30383331,697,2021-10-01,N,2036-09-01,12060.0,0,1,P,64,...,NaN,NaN,NaN,37,NaN,NaN,NaN,NaN,NaN,207054.23
3,F21Q30383332,667,2021-10-01,N,2036-09-01,33874.0,0,1,P,55,...,NaN,NaN,NaN,29,NaN,NaN,NaN,NaN,NaN,61525.27
4,F21Q30383333,698,2021-10-01,N,2051-09-01,45780.0,0,1,P,67,...,NaN,NaN,NaN,48,NaN,NaN,NaN,NaN,NaN,100177.64


In [16]:
df1 = df1.rename(columns={
    "Unnamed: 0": "Loan Sequence Number", 
})
df2 = df2.rename(columns={
    "Unnamed: 0": "Loan Sequence Number", 
})
df1 = df1[~df1["Loan Sequence Number"].astype(str).str.startswith("F19")]
df1.head()

,Loan Sequence Number,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),...,Modification Cost,Step Modification Flag,Deferred Payment Plan,Estimated Loan-to-Value (ELTV),Zero Balance Removal UPB,Delinquent Accrued Interest,Delinquency Due to Disaster,Borrower Assistance Status Code,Current Month Modification Cost,Interest Bearing UPB
1251476,F20Q10000001,661,2020-06-01,N,2035-05-01,41540.0,0,1,P,36,...,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,40665.26
1251477,F20Q10000002,681,2020-03-01,N,2050-02-01,45820.0,30,1,P,95,...,NaN,NaN,NaN,34,NaN,NaN,NaN,NaN,NaN,46807.70
1251478,F20Q10000003,775,2020-04-01,N,2050-03-01,NaN,25,1,P,87,...,NaN,NaN,NaN,999,238153.92,NaN,NaN,NaN,NaN,0.00
1251479,F20Q10000004,770,2020-03-01,N,2035-02-01,41180.0,0,2,I,65,...,NaN,NaN,NaN,999,NaN,NaN,NaN,NaN,NaN,89979.70
1251480,F20Q10000005,791,2020-04-01,N,2050-03-01,10580.0,0,1,P,80,...,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,47698.13


In [17]:
df = pd.concat([df1,df2],axis=0)

In [18]:
df.head()

,Loan Sequence Number,Credit Score,First Payment Date,First Time Homebuyer Flag,Maturity Date,Metropolitan Statistical Area (MSA) Or Metropolitan Division,Mortgage Insurance Percentage (MI %),Number of Units,Occupancy Status,Original Combined Loan-to-Value (CLTV),...,Modification Cost,Step Modification Flag,Deferred Payment Plan,Estimated Loan-to-Value (ELTV),Zero Balance Removal UPB,Delinquent Accrued Interest,Delinquency Due to Disaster,Borrower Assistance Status Code,Current Month Modification Cost,Interest Bearing UPB
1251476,F20Q10000001,661,2020-06-01,N,2035-05-01,41540.0,0,1,P,36,...,NaN,NaN,NaN,14,NaN,NaN,NaN,NaN,NaN,40665.26
1251477,F20Q10000002,681,2020-03-01,N,2050-02-01,45820.0,30,1,P,95,...,NaN,NaN,NaN,34,NaN,NaN,NaN,NaN,NaN,46807.70
1251478,F20Q10000003,775,2020-04-01,N,2050-03-01,NaN,25,1,P,87,...,NaN,NaN,NaN,999,238153.92,NaN,NaN,NaN,NaN,0.00
1251479,F20Q10000004,770,2020-03-01,N,2035-02-01,41180.0,0,2,I,65,...,NaN,NaN,NaN,999,NaN,NaN,NaN,NaN,NaN,89979.70
1251480,F20Q10000005,791,2020-04-01,N,2050-03-01,10580.0,0,1,P,80,...,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,47698.13


In [19]:
# Selected column names for logistic regression
selected_columns = [
    'Loan Sequence Number',
    'Credit Score',
    'Original Loan-to-Value (LTV)',
    'Original Combined Loan-to-Value (CLTV)',
    'Original Debt-to-Income (DTI) Ratio',
    'Original Interest Rate',
    'Original UPB',
    'Current Actual UPB',
    'Loan Age',
    'Remaining Months to Legal Maturity',
    'Estimated Loan-to-Value (ELTV)',
    'Current Loan Delinquency Status',
    'Number of Borrowers',
    'Property State',
    'Current Deferred UPB',
    'Current Interest Rate',
    'Occupancy Status',
    'Original Loan Term',
    'First Payment Date',
]
df = df[selected_columns]
df.head()

,Loan Sequence Number,Credit Score,Original Loan-to-Value (LTV),Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,Original Interest Rate,Original UPB,Current Actual UPB,Loan Age,Remaining Months to Legal Maturity,Estimated Loan-to-Value (ELTV),Current Loan Delinquency Status,Number of Borrowers,Property State,Current Deferred UPB,Current Interest Rate,Occupancy Status,Original Loan Term,First Payment Date
1251476,F20Q10000001,661,36,36,19,2.875,66000,40665.26,58,122,14,0,2,MD,0.0,2.875,P,180,2020-06-01
1251477,F20Q10000002,681,95,95,13,5.750,52000,46807.70,61,299,34,0,1,KS,0.0,5.750,P,360,2020-03-01
1251478,F20Q10000003,775,87,87,29,3.250,248000,0.00,24,336,999,0,2,CO,0.0,3.250,P,360,2020-04-01
1251479,F20Q10000004,770,65,65,14,3.625,125000,89979.70,61,119,999,0,1,MO,0.0,3.625,I,180,2020-03-01
1251480,F20Q10000005,791,80,80,33,3.875,58000,47698.13,60,300,20,0,1,NY,0.0,3.875,P,360,2020-04-01


In [20]:
# Derive binary target 'Default' from 'Current Loan Delinquency Status'
if "Current Loan Delinquency Status" in df.columns:
    delinquency = pd.to_numeric(df["Current Loan Delinquency Status"], errors="coerce").fillna(0)

    df["Default"] = np.where(
        (delinquency >= 3) | (df["Current Loan Delinquency Status"].astype(str) == "RA"),1, 0
)
df.head()

,Loan Sequence Number,Credit Score,Original Loan-to-Value (LTV),Original Combined Loan-to-Value (CLTV),Original Debt-to-Income (DTI) Ratio,Original Interest Rate,Original UPB,Current Actual UPB,Loan Age,Remaining Months to Legal Maturity,Estimated Loan-to-Value (ELTV),Current Loan Delinquency Status,Number of Borrowers,Property State,Current Deferred UPB,Current Interest Rate,Occupancy Status,Original Loan Term,First Payment Date,Default
1251476,F20Q10000001,661,36,36,19,2.875,66000,40665.26,58,122,14,0,2,MD,0.0,2.875,P,180,2020-06-01,0
1251477,F20Q10000002,681,95,95,13,5.750,52000,46807.70,61,299,34,0,1,KS,0.0,5.750,P,360,2020-03-01,0
1251478,F20Q10000003,775,87,87,29,3.250,248000,0.00,24,336,999,0,2,CO,0.0,3.250,P,360,2020-04-01,0
1251479,F20Q10000004,770,65,65,14,3.625,125000,89979.70,61,119,999,0,1,MO,0.0,3.625,I,180,2020-03-01,0
1251480,F20Q10000005,791,80,80,33,3.875,58000,47698.13,60,300,20,0,1,NY,0.0,3.875,P,360,2020-04-01,0


In [21]:
df["Default"].value_counts()

Default
0    11524257
1       45558
Name: count, dtype: int64

In [22]:
output_fp = "/Users/61310joy/Default_Predict/Data/regression_data/regression.csv"
df.to_csv(output_fp, index=False)